<a href="https://colab.research.google.com/github/rautaditya2606/Santander-Customer-Transaction-Prediction/blob/main/santander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px
import matplotlib
%matplotlib inline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import opendatasets as od
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_text
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
from datetime import datetime
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor, XGBClassifier
import joblib
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier



sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10,6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
database_url = 'https://www.kaggle.com/competitions/santander-customer-transaction-prediction/overview'
od.download(database_url)

Skipping, found downloaded files in "./santander-customer-transaction-prediction" (use force=True to force download)


In [ ]:
data_dir = '/content/santander-customer-transaction-prediction'

In [ ]:
raw_df = pd.read_csv('/content/santander-customer-transaction-prediction/train.csv')
test_df = pd.read_csv('/content/santander-customer-transaction-prediction/test.csv')
submission_df = pd.read_csv('/content/santander-customer-transaction-prediction/sample_submission.csv')

In [ ]:
raw_df

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,6.1415,13.2305,3.9901,0.9388,18.0249,-1.7939,2.1661,8.5326,16.6660,-17.8661
199996,train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,4.9611,4.6549,0.6998,1.8341,22.2717,1.7337,-2.1651,6.7419,15.9054,0.3388
199997,train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,4.0651,5.4414,3.1032,4.8793,23.5311,-1.5736,1.2832,8.7155,13.8329,4.1995
199998,train_199998,0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,2.6840,8.6587,2.7337,11.1178,20.4158,-0.0786,6.7980,10.0342,15.5289,-13.9001


In [ ]:
# raw_df.corr(numeric_only=True)['target'].sort_values(ascending=False)

In [ ]:
raw_df.select_dtypes(include=['float', 'int']).columns

Index(['target', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6',
       'var_7', 'var_8',
       ...
       'var_190', 'var_191', 'var_192', 'var_193', 'var_194', 'var_195',
       'var_196', 'var_197', 'var_198', 'var_199'],
      dtype='object', length=201)

In [ ]:
inputs_cols = raw_df.columns.drop(['ID_code', 'target'])
target_col = 'target'

In [ ]:
inputs = raw_df[inputs_cols]
target = raw_df[target_col]

In [ ]:
train_inputs, val_inputs, train_target, val_target = train_test_split(inputs, target, test_size=0.3, random_state=42, stratify=target)

In [ ]:
# !pip install catboost

In [ ]:
scaler = MinMaxScaler().fit(train_inputs)

In [ ]:
train_inputs_scaled = scaler.transform(train_inputs)

In [ ]:
val_inputs_scaled = scaler.transform(val_inputs)
test_inputs = test_df[inputs_cols]
test_inputs_scaled = scaler.transform(test_inputs)

In [ ]:
# param_dict = {
#     'max_depth' : [3,4,5,6,7,8,10],
#     'learning_rate' : [0.05, 0.1, 0.15, 0.2],
#     'n_estimators' : [300, 500, 1000],
#     'subsample': [0.6, 0.8, 1.0],
#     'comsample_bytree': [0.6, 0.8, 1.0],
#     'min_child_weight': [1,3,5,7,9],
#     'gamma': [0, 0.1, 0.5, 1, 2],
#     'reg_alpha': [0, 0.1, 1, 2],
#     'reg_lambda': [0, 0.1, 1, 2]
# }

# search = RandomizedSearchCV(
#     XGBClassifier(
#         n_job=-1,
#         eval_metric= 'auc',
#         use_label_encoder = False,
#     ),
#     param_distributions = param_dict,
#     n_iter = 30,
#     scoring='roc_auc',
#     cv=3,
#     verbose = 2,
#     random_state=42
# )

# search.fit(train_inputs, train_target)
# print('Best Param: ', search.best_params_)
train_inputs

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
59864,9.6933,-2.4255,10.9351,4.7861,11.4639,-1.9561,5.0136,10.0195,-2.6873,9.6778,...,9.3910,4.4830,-0.5747,6.0496,12.8398,0.1474,11.3712,10.4911,16.7409,2.5494
128990,9.0945,-3.4072,8.3412,4.1683,12.9005,-1.0546,6.2069,19.1833,4.4601,5.8213,...,6.7898,3.8164,3.5985,0.6460,18.8068,0.3827,-0.7915,10.0788,21.9431,-15.6626
27890,4.8086,1.7615,6.4716,7.1207,12.7379,-14.5603,4.0059,12.2351,-1.9882,8.8706,...,3.5133,11.1094,3.9747,6.8936,14.9802,-1.5616,-0.1406,8.2615,16.7520,-1.6382
143056,9.4214,2.7131,15.6923,4.3296,11.4713,-8.3192,5.8888,20.7898,1.9752,7.3627,...,6.9628,5.4222,1.0913,6.6895,19.8162,0.0387,-3.3236,8.2064,16.7587,-28.6671
4271,9.9651,1.1076,11.4266,6.3137,9.6954,-6.0839,3.8754,17.1563,2.4501,8.9588,...,4.8282,7.7861,3.1748,-1.7787,22.2981,-2.8483,10.3890,7.7566,17.5830,-15.5327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160482,8.0801,-2.5523,10.6587,10.6660,8.7552,-10.8144,5.2019,21.1479,0.4870,7.8673,...,4.9773,7.5440,0.8520,4.9402,19.0033,1.5157,-9.3063,9.0586,14.9677,-11.2478
48966,11.8371,-4.9598,12.9172,9.3759,13.0622,2.6849,5.2963,19.8298,3.6545,7.5124,...,2.3344,13.5478,3.1656,-1.5778,17.3154,-0.9673,4.9180,8.8015,13.2019,-21.5102
52242,11.3368,1.2790,14.1568,11.9850,11.4859,0.2191,5.9112,19.7031,2.4183,5.9524,...,5.7440,11.3343,2.8999,4.8399,14.0429,0.5310,-0.0909,6.7786,13.0894,-0.6329
7767,3.4660,-0.2570,4.3530,7.2045,11.4988,-7.0838,5.5081,13.7160,-1.0089,8.7558,...,2.3471,2.6189,4.6409,9.0566,15.5558,-0.0071,1.0802,9.6420,14.3480,4.5444


In [ ]:
# import xgboost as xgb
# from sklearn.metrics import roc_auc_score

# dtrain = xgb.DMatrix(train_inputs, label=train_target)
# dval = xgb.DMatrix(val_inputs, label=val_target)

# params = {
#     'objective': 'binary:logistic',
#     'eval_metric': 'auc',
#     'learning_rate': 0.01,
#     'max_depth': 3,
#     'min_child_weight': 10,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'lambda': 20,
#     'alpha': 10,
#     'scale_pos_weight': (len(train_target) - sum(train_target)) / sum(train_target),
#     'seed': 42,
#     'verbosity': 1,
#     'n_estimators' : 500
# }

# evals = [(dtrain, 'train'), (dval, 'eval')]

# model_xgb = xgb.train(
#     params,
#     dtrain,
#     num_boost_round=2000,
#     evals=evals,
#     early_stopping_rounds=100,
#     verbose_eval=100
# )

# train_pred = model_xgb.predict(dtrain)
# val_pred = model_xgb.predict(dval)

# print('Train AUC:', roc_auc_score(train_target, train_pred))
# print('Validation AUC:', roc_auc_score(val_target, val_pred))


In [ ]:
model = XGBClassifier(n_jobs=-1, n_estimators = 500, max_depth = 4, learning_rate = 0.05).fit(train_inputs_scaled, train_target)

In [ ]:
train_prob = model.predict_proba(train_inputs_scaled)[:,-1]
val_prob = model.predict_proba(val_inputs_scaled)[:,-1]

In [ ]:
print('Train AUC', roc_auc_score(train_target, train_prob))
print('Val AUC', roc_auc_score(val_target, val_prob))#Val AUC 0.8927424300424831

Train AUC 0.9400639190351557
Val AUC 0.8771309840658579


In [ ]:
test_pred = model.predict(test_inputs_scaled)
test_pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
submission_df['target'] = test_pred

In [ ]:
submission_df

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0
...,...,...
199995,test_199995,0
199996,test_199996,0
199997,test_199997,0
199998,test_199998,0


In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

inputs_cols = raw_df.columns.drop(['ID_code', 'target'])
X = raw_df[inputs_cols].values
y = raw_df['target'].values

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

val_aucs = []
test_preds = np.zeros(len(test_df))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    scaler = MinMaxScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(test_df[inputs_cols].values)

    dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
    dval = xgb.DMatrix(X_val_scaled, label=y_val)
    dtest = xgb.DMatrix(X_test_scaled)

    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'learning_rate': 0.03,
        'max_depth': 5,
        'min_child_weight': 10,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'lambda': 20,
        'alpha': 10,
        'scale_pos_weight': (len(y_train) - sum(y_train)) / sum(y_train),
        'seed': 42,
        'verbosity': 0,
    }

    evals = [(dtrain, 'train'), (dval, 'eval')]
    model = xgb.train(
        params,
        dtrain,
        num_boost_round=2000,
        evals=evals,
        early_stopping_rounds=100,
        verbose_eval=100,
    )

    val_pred = model.predict(dval)
    val_auc = roc_auc_score(y_val, val_pred)
    val_aucs.append(val_auc)

    test_preds += model.predict(dtest) / skf.n_splits

    print(f'Fold {fold} Validation AUC: {val_auc:.5f}')

print(f'Mean Validation AUC: {np.mean(val_aucs):.5f} ± {np.std(val_aucs):.5f}')

submission_df['target'] = test_preds
submission_df.to_csv('submission_cv.csv', index=False)


[0]	train-auc:0.63152	eval-auc:0.62183
[100]	train-auc:0.84298	eval-auc:0.80307
[200]	train-auc:0.88613	eval-auc:0.83750
[300]	train-auc:0.90863	eval-auc:0.85375
[400]	train-auc:0.92339	eval-auc:0.86460
[500]	train-auc:0.93398	eval-auc:0.87109
[600]	train-auc:0.94215	eval-auc:0.87598
[700]	train-auc:0.94822	eval-auc:0.87982
[800]	train-auc:0.95350	eval-auc:0.88277
[900]	train-auc:0.95785	eval-auc:0.88514
[1000]	train-auc:0.96167	eval-auc:0.88699
[1100]	train-auc:0.96478	eval-auc:0.88842
[1200]	train-auc:0.96761	eval-auc:0.88961
[1300]	train-auc:0.97002	eval-auc:0.89039
[1400]	train-auc:0.97233	eval-auc:0.89133
[1500]	train-auc:0.97442	eval-auc:0.89179
[1600]	train-auc:0.97635	eval-auc:0.89223
[1700]	train-auc:0.97799	eval-auc:0.89262
[1800]	train-auc:0.97955	eval-auc:0.89304
[1900]	train-auc:0.98098	eval-auc:0.89349
[1999]	train-auc:0.98231	eval-auc:0.89381
Fold 1 Validation AUC: 0.89381
[0]	train-auc:0.62727	eval-auc:0.62270
[100]	train-auc:0.84413	eval-auc:0.81021
[200]	train-auc:0.8